In [1]:
import warnings
warnings.filterwarnings('ignore')
from  Model_test import *

In [2]:
import pandas as pd
pd.options.display.max_columns = 75
%pylab
%matplotlib inline

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.read_csv('dataset_diabetes/diabetic_data.csv')

In [4]:
df.head(25)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,?,?,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),?,3,1,2,4,?,?,70,1,21,0,0,0,414,411,V45,7,None,None,Steady,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),?,1,1,7,5,?,?,73,0,12,0,0,0,428,492,250,8,None,None,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),?,2,1,4,13,?,?,68,2,28,0,0,0,398,427,38,8,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),?,3,3,4,12,?,InternalMedicine,33,3,18,0,0,0,434,198,486,8,None,None,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [5]:
import pandas_profiling 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        101766 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  101766 non-null object
medical_specialty           101766 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            10176

In [7]:
df.insulin.head(25)

0         No
1         Up
2         No
3         Up
4     Steady
5     Steady
6     Steady
7         No
8     Steady
9     Steady
10    Steady
11    Steady
12      Down
13    Steady
14    Steady
15        Up
16    Steady
17        No
18    Steady
19    Steady
20      Down
21    Steady
22        No
23        No
24    Steady
Name: insulin, dtype: object

In [8]:
pandas_profiling.ProfileReport(df)

Number of variables,50
Number of observations,101766
Total Missing (%),0.0%
Total size in memory,38.8 MiB
Average record size in memory,400.0 B
Numeric,13
Categorical,35
Boolean,0
Date,0
Text (Unique),0
Rejected,2


## Cleaning some data


In [9]:
# Drop becuase all values are the same
df.drop(columns = ['citoglipton','weight','examide','encounter_id'],inplace=True)

# weight, drop coz 97% missing
# citoglipton, and examide are constant values
# encounter_id is just an id

In [10]:
# race, missing set to other 2%
df.race.replace('?', 'Other',inplace=True)


In [11]:
df.drop(columns = ['diag_1', 'diag_2','diag_3','payer_code'],inplace=True)


In [12]:
# convert diabetes_med from Yes or No to 1 or 0
# convert readmission from No, <30, >30 to 0, 1, 2


In [13]:
# convert change from No, Ch to 0, 1
df.change = np.where(df.change=='No',0,1)

In [14]:
# convert diabetes_med from Yes or No to 1 or 0
df.diabetesMed = np.where(df.diabetesMed=='Yes',1,0)

In [15]:
df.drop('medical_specialty', axis=1, inplace=True)
# 49,000 are ?

In [16]:
df.max_glu_serum.unique()

array(['None', '>300', 'Norm', '>200'], dtype=object)

In [17]:
df.readmitted = np.where(df.readmitted=='NO', 0, df.readmitted )
df.readmitted = np.where(df.readmitted=='<30', 1,  df.readmitted )
df.readmitted = np.where(df.readmitted=='>30', 2,  df.readmitted )

In [18]:
# df.patient_nbr.nunique()
# plt.scatter(df.patient_nbr, df.encounter_id, alpha=0.2)
df.sort_values(by='patient_nbr').head()

,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
4267,135,Caucasian,Female,[50-60),2,1,1,8,77,6,33,0,0,0,8,None,None,Steady,No,No,No,No,No,No,Down,No,No,No,No,No,No,No,Steady,No,No,No,No,No,1,1,1
4780,135,Caucasian,Female,[50-60),1,1,7,3,31,1,14,0,0,1,5,None,None,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,1,1,2
5827,378,Caucasian,Female,[50-60),3,1,1,2,49,1,11,0,0,0,3,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0,0,0
67608,729,Caucasian,Female,[80-90),1,3,7,4,68,2,23,0,0,0,9,None,>7,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0,1,0
17494,774,Caucasian,Female,[80-90),1,1,7,3,46,0,20,0,0,0,9,None,>8,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,1,1,0


In [19]:
dummy_df = pd.get_dummies(df,drop_first=True)

In [20]:
dummy_df.drop(columns=['readmitted_1', 'readmitted_2','gender_Unknown/Invalid'],
              inplace=True)

In [21]:
X = dummy_df.copy()
y = df.readmitted.astype(int)

## Train Test Split

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
# X_train, X_test, y_train, y_test = train_test_split(X,y,  random_state=42, 
#                                                     test_size=0.3)

In [27]:
X_train, X_test, y_train, y_test = scale_train_test_split(X,y)

In [28]:
y_test

69739     2
55987     2
49399     2
12195     2
6774      2
54983     2
68977     2
100329    2
45994     2
679       0
56682     2
27829     0
64308     0
92321     0
18120     0
48351     1
32262     0
31414     0
71702     0
84004     2
19271     0
47956     0
40729     2
70082     0
40248     0
70378     1
10377     2
22191     2
55932     0
2657      2
         ..
91338     0
38865     1
30564     0
65553     2
39214     1
35560     0
101402    2
66219     0
54137     1
46034     0
52212     2
12674     0
73358     2
87323     0
68722     0
14791     2
47506     0
65238     2
54146     0
92222     2
100665    0
12936     2
71157     0
61552     0
77271     2
34679     0
95652     0
60637     0
44538     1
38254     2
Name: readmitted, Length: 25442, dtype: int64

In [29]:
models, model_names = train_model(X_train, y_train)

In [30]:
y_train_predicts = predict_all(X_train, y_train, models)

In [31]:
y_test_predicts = predict_all(X_test, y_test, models)

['adaboost', 'gbt']

In [ ]:
type(y_train_predicts[1])

In [ ]:
type(y_train)

In [39]:

print('--- Train data ---')
for i,y_train_pred in enumerate(y_train_predicts):
    display_acc_and_f1_score(y_train, y_train_pred, model_names[i])
print('--- Test data ---')
for i,y_test_pred in enumerate(y_test_predicts):
    display_acc_and_f1_score(y_test, y_test_pred, model_names[i])

--- Train data ---
Model: adaboost
Accuracy: 0.5846784759708611
F1-Score: 0.5846784759708611
Model: gbt
Accuracy: 0.5936009643100466
F1-Score: 0.5936009643100466
--- Test data ---
Model: adaboost
Accuracy: 0.5850955113591699
F1-Score: 0.5850955113591699
Model: gbt
Accuracy: 0.5873752063517019
F1-Score: 0.5873752063517019


In [34]:
for i,y_train_pred in enumerate(y_train_predicts):
    display_acc_and_f1_score(np.array(y_train), y_train_pred, model_names[i])

0.5846784759708611

In [38]:
f1_score(y_train, y_train_predicts[0],average='weighted')

0.5324889321412017

## Base Model

## Columns to inspect

    - Admission_type_id
    - discharge_disposition_id
    - admission_source_id
    - number_of_lab_procedures
    